## K-means Clustering
The intent of this section is to show the results of k-means clustering
We will use the Iris data set. I will color the type of Iris for demonstration purposes, but remember that K-means is unsupervised

In [ ]:
# All my imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from fcmeans import FCM
from sklearn import datasets
from sklearn.decomposition import PCA
import seaborn as sns

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target


Pair plots can help us in our data exploration phase
We can visualize how clusters might look 
When the pair plot is with itself, it shows a distribution on that variable.
Look at  Sepal Width (assuming I properly mapped the columns). 
Does it add much value?

In [ ]:
# Lets do some pair plots of the features
df = pd.DataFrame(X)
df.columns = ["Sepal Length","Sepal Width","Pedal Length","Pedal Width"]
df["Species"] = y
sns.pairplot(df,hue="Species")

From the pair plots we can get an idea on which variables contribute the most. We could likely removed Sepa Width. If we wanted to keep two original varialbes, the pedal variables seem to provide the most information.
However, we want to do a demo of PCA ... so lets look at that.

In [ ]:
# PCA Code - Down to 3 dimensions at first, just for example
X2 = X.copy()
pca = PCA(n_components=3)
pca.fit(X2)
X2 = pca.transform(X2)
df2 = pd.DataFrame(X2)
df2.columns = ["Component1","Component2","Component3"]
df2["Species"] = y
print(df2.describe())
print(df2.head())

In [ ]:
# https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_iris.html
import warnings
warnings.filterwarnings("ignore")
# Warning about deprecated methods were annoying me
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(1, figsize=(4, 3))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
for name, label in [('Setosa', 0), ('Versicolour', 1), ('Virginica', 2)]:
    ax.text3D(X2[y == label, 0].mean(),
              X2[y == label, 1].mean() + 1.5,
              X2[y == label, 2].mean(), name,
              horizontalalignment='center',
              bbox=dict(alpha=.5, edgecolor='w', facecolor='w'))
# Reorder the labels to have colors matching the cluster results
y = np.choose(y, [1, 2, 0]).astype(float)
ax.scatter(X2[:, 0], X2[:, 1], X2[:, 2], c=y, cmap=plt.cm.nipy_spectral,
           edgecolor='k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

Now we are down to 3 dimensions, we can visualize it better than 4 dimensions
We can see a nice separation for Setosa, but the other two overlap a bit

We will set n_init to 1 for one test. 
The n_init variable controls how many times it will initialize with different centroids
The implemention of kmeans in scikitlearn will run the algorithm with n_init different starts and pick the model that it thinks is best based on an inertia meteric.
Inertia is calculated uses sum of squares to closest cluster center.
It is possible that the first choice is the best choice and n_init = 1 could have good results. The default is 10.

In [ ]:
# https://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_iris.html


estimators = [('k_means_iris_8', KMeans(n_clusters=8)),
              ('k_means_iris_3', KMeans(n_clusters=3)),
              ('k_means_iris_bad_init', KMeans(n_clusters=3, n_init=1,
                                               init='random'))]

fignum = 1
titles = ['8 clusters', '3 clusters', '3 clusters, bad initialization']
for name, est in estimators:
    fig = plt.figure(fignum, figsize=(4, 3))
    ax = Axes3D(fig, rect=[0, 0, 2, 2], elev=48, azim=134)
    est.fit(X)
    labels = est.labels_

    ax.scatter(X2[:, 0], X2[:, 1], X2[:, 2],
               c=labels.astype(float), edgecolor='k')

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])
    ax.set_xlabel('Component 1')
    ax.set_ylabel('Component 2')
    ax.set_zlabel('Component 3')
    ax.set_title(titles[fignum - 1])
    ax.dist = 12
    fignum = fignum + 1

# Plot the ground truth
fig = plt.figure(fignum, figsize=(4, 3))
ax = Axes3D(fig, rect=[0, 0, 2, 2], elev=48, azim=134)

for name, label in [('Setosa', 1),
                    ('Versicolour', 2),
                    ('Virginica', 0)]:
    ax.text3D(X2[y == label, 0].mean(),
              X2[y == label, 1].mean(),
              X2[y == label, 2].mean() + 2, name,
              horizontalalignment='center',
              bbox=dict(alpha=.2, edgecolor='w', facecolor='w'))
# Reorder the labels to have colors matching the cluster results
#y = np.choose(y, [1, 2, 0]).astype(int)
ax.scatter(X2[:, 0], X2[:, 1], X2[:, 2], c=y, edgecolor='k')

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
ax.set_zlabel('Component 3')
ax.set_title('Ground Truth')
ax.dist = 12

fig.show()

## K Nearest Neighbor


In [ ]:
# Moving to 2D because it is easier for me to visualize
# and that is the code I am reusing
# https://scikit-learn.org/stable/auto_examples/neighbors/plot_classification.html

# I am pulling the data back in because I copied the code mostly verbatim
from matplotlib.colors import ListedColormap
from sklearn import neighbors

n_neighbors = 15

# import some data to play with
iris = datasets.load_iris()

# we only take the first two features. We could avoid this ugly
# slicing by using a two-dim dataset
X = iris.data[:, :2]
y = iris.target

h = .02  # step size in the mesh

# Create color maps
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue'])
cmap_bold = ['darkorange', 'c', 'darkblue']

for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(X, y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=iris.target_names[y],
                    palette=cmap_bold, alpha=1.0, edgecolor="black")
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("3-Class classification (k = %i, weights = '%s')"
              % (n_neighbors, weights))
    plt.xlabel(iris.feature_names[0])
    plt.ylabel(iris.feature_names[1])

plt.show()

In [ ]:
# Lets do it again, but use pedal features
# We could also use PCA, but then we lose understanding of the relationship between variables

# we only take the first two features. We could avoid this ugly
# slicing by using a two-dim dataset
X = iris.data[:, 2:]
y = iris.target

h = .02  # step size in the mesh

# Create color maps
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue'])
cmap_bold = ['darkorange', 'c', 'darkblue']

for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(X, y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=iris.target_names[y],
                    palette=cmap_bold, alpha=1.0, edgecolor="black")
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("3-Class classification (k = %i, weights = '%s')"
              % (n_neighbors, weights))
    plt.xlabel(iris.feature_names[2])
    plt.ylabel(iris.feature_names[3])

plt.show()

I dont know about you, but I think this looks a lot better when we use pedal variables
I am curious what it will look like if I use PCA to 2 variables


In [ ]:
X = iris.data
pca = PCA(n_components=2)
pca.fit(X)
X = pca.transform(X)
y = iris.target

h = .02  # step size in the mesh

# Create color maps
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue'])
cmap_bold = ['darkorange', 'c', 'darkblue']

for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(X, y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=iris.target_names[y],
                    palette=cmap_bold, alpha=1.0, edgecolor="black")
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title("3-Class classification (k = %i, weights = '%s')"
              % (n_neighbors, weights))
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")

plt.show()

We can see that the results are similar to using just the petal features, but not quite.
We also see that the domain of the components are not the same as the original variables.


## Support Vector Machines
I am going to start with the reduced dimensions from the previous block
This dataset is clean enough that I don't expect to need to use anything other than the default settings.
I didn't cover the predict method for k-nearest neighbor but they all work about the same

The scikitlearn documentation page uses Sepal measurements. My guess is that they did it because it makes a more interesting example 

https://scikit-learn.org/stable/modules/svm.html

In [ ]:
# Too easy
from sklearn import svm
clf = svm.SVC()
clf.fit(X, y)

# Lets test a prediction
print("The prediction is",iris.target_names[clf.predict([[2., 2.]])][0])
# We could look at other things such as predict_proba if we want to see other classes
clf = svm.SVC(probability=True)
clf.fit(X, y)
probs = clf.predict_proba([[2., 2.]])
print("The probability array is",probs)
# Now we can see the estimated probabilty of each class. 
# Maybe I should clean up the results
print("Probability of {} is {:0.2f}%".format(iris.target_names[0],probs[0][0]*100))
print("Probability of {} is {:0.2f}%".format(iris.target_names[1],probs[0][1]*100))
print("Probability of {} is {:0.2f}%".format(iris.target_names[2],probs[0][2]*100))

In [ ]:
# Plot it
# https://towardsdatascience.com/support-vector-machines-svm-clearly-explained-a-python-tutorial-for-classification-problems-29c539f3ad8
# BTW, this is a good article. 
# It is much better than the one I used when I was writing the slides
# ... but I just found it while working on the demo notebook


feature_names = ["Component 1","Component 2"]
classes = iris.target_names
def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy
def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out
# The classification SVC model
model = svm.SVC(kernel="linear")
clf = model.fit(X, y)
fig, ax = plt.subplots()
# title for the plots
title = ('Decision surface of linear SVC')
# Set-up grid for plotting.
X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)
plot_contours(ax, clf, xx, yy, cmap=plt.cm.coolwarm, alpha=0.8)
ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors="k")
ax.set_ylabel("{}".format(feature_names[0]))
ax.set_xlabel("{}".format(feature_names[1]))
ax.set_xticks(())
ax.set_yticks(())
ax.set_title(title)
plt.show()

In [ ]:
# 3D example
X = iris.data
pca = PCA(n_components=3)
pca.fit(X)
X = pca.transform(X)
Y = iris.target

#make it binary classification problem
X = X[np.logical_or(Y==0,Y==1)]
Y = Y[np.logical_or(Y==0,Y==1)]
model = svm.SVC(kernel='linear')
clf = model.fit(X, Y)
# The equation of the separating plane is given by all x so that np.dot(svc.coef_[0], x) + b = 0.
# Solve for w3 (z)
z = lambda x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]
tmp = np.linspace(-1,1,2)
x,y = np.meshgrid(tmp,tmp)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot3D(X[Y==0,0], X[Y==0,1], X[Y==0,2],'ob')
ax.plot3D(X[Y==1,0], X[Y==1,1], X[Y==1,2],'sr')
ax.plot_surface(x, y, z(x,y))
ax.view_init(15,45)
plt.show()



In [ ]:
# A new example with random points
# Show the margin
# Show the support vectors

# Set seed so we always get the same results.
# Maybe change and run again
np.random.seed(2)

# we create 40 linearly separable points
X = np.r_[np.random.randn(20, 2) - [2, 2], np.random.randn(20, 2) + [2, 2]]
Y = [0] * 20 + [1] * 20
# fit the model
clf = svm.SVC(kernel='linear', C=1)
clf.fit(X, Y)

# get the separating hyperplane
w = clf.coef_[0]
a = -w[0] / w[1]
# Get a line for x 
xx = np.linspace(-5, 5)
# Build the line for yy using xx and the model
yy = a * xx - (clf.intercept_[0]) / w[1]

# Calculate the margine using coefficient in the model
## Sq Root of Sum of squares
margin = 1 / np.sqrt(np.sum(clf.coef_ ** 2))
# Offset the separator line by the margin
yy_down = yy - np.sqrt(1 + a ** 2) * margin
yy_up = yy + np.sqrt(1 + a ** 2) * margin

plt.figure(1, figsize=(4, 3))
plt.clf()
plt.plot(xx, yy, "k-")
plt.plot(xx, yy_down, "k-")
plt.plot(xx, yy_up, "k-")
# The first scatter plot grabs just the points on the line and makes them stand out
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=80,
 facecolors="none", zorder=10, edgecolors="k")

# Plot all the points
plt.scatter(X[:, 0], X[:, 1], c=Y, zorder=10, cmap=plt.cm.Paired,
 edgecolors="k")
plt.xlabel("x1")
plt.ylabel("x2")
plt.show()